## 00. 프로젝트 목적
- 본 프로젝트는 Pinecone vector store 업로드를 위한 프로젝트입니다.
- 한번에 한 파일만 올리는 것이 좋으며, namespace & metadata 전략을 적절히 사용하는 것이 유리합니다.

- [Pinecone 공식 홈페이지](https://docs.pinecone.io/integrations/langchain)
- [Pinecone 랭체인](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/)

### 필요한 환경변수 로드

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API 키 정보 로드
load_dotenv()

# API 키 읽어오기
openai_api_key = os.environ.get('OPENAI_API_KEY')
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

## 01. PDF to RAG Chunks

- `load_and_process_docs`: PDF 청킹 함수
    - 필요한 메타데이터만 추출하여 함수로 저장함.
        - headings: 추출한 객체의 대주제
        - doc_items_labels: text인지, list인지, table인지 구분
        - filename: 업로드 한 파일명

In [11]:
from langchain_docling import DoclingLoader
from docling.chunking import HybridChunker
from langchain_docling.loader import ExportType
from langchain_core.documents import Document

# 문서 로딩 및 청킹을 위한 설정
FILE_PATH = ["../create_dataset/source_data/prompt_test.pdf"]
EXPORT_TYPE = ExportType.DOC_CHUNKS # 문서 청크 추출 타입 설정
EMBED_MODEL_ID = "BAAI/bge-m3"

def load_and_process_docs(file_path, embed_model_id, chunk_size=1000, overlap=100):
    """
    문서를 로드하고 Pinecone 호환 메타데이터로 처리하는 함수
    
    Args:
        file_path (list): 로드할 파일 경로 리스트
        embed_model_id (str): 임베딩 모델 ID
        chunk_size (int, optional): 청크 크기. 기본값 1200
        overlap (int, optional): 청크 오버랩. 기본값 100
        
    Returns:
        list: Pinecone 호환 메타데이터를 가진 Document 객체 리스트
    """
    # DoclingLoader 초기화 및 로딩
    loader = DoclingLoader(
        file_path=file_path,
        export_type=ExportType.DOC_CHUNKS,
        chunker=HybridChunker(tokenizer=embed_model_id, chunk_size=chunk_size, overlap=overlap)
    )
    
    # 문서 로드
    original_docs = loader.load()
    print(f"로드된 문서 수: {len(original_docs)}")
    
    # 메타데이터 변환 및 새 문서 생성
    pinecone_docs = []
    
    for doc in original_docs:
        # 메타데이터에서 필요한 정보만 추출
        simplified = {}
        metadata = doc.metadata
        
        # 1. dl_meta의 doc_items의 label 추출
        if 'dl_meta' in metadata and isinstance(metadata['dl_meta'], dict):
            # headings 추출
            if 'headings' in metadata['dl_meta']:
                simplified['headings'] = metadata['dl_meta']['headings']
                
            # doc_items의 label 추출
            if 'doc_items' in metadata['dl_meta'] and metadata['dl_meta']['doc_items']:
                items = metadata['dl_meta']['doc_items']
                labels = [item.get('label') for item in items if 'label' in item]
                if labels:
                    simplified['doc_items_labels'] = labels

        # 2. origin의 filename 추출
        if ('dl_meta' in metadata and 'origin' in metadata['dl_meta'] and 
                'filename' in metadata['dl_meta']['origin']):
            simplified['filename'] = metadata['dl_meta']['origin']['filename']
        
        # 페이지 번호 추가 (있을 경우)
        if 'page' in metadata:
            simplified['page'] = metadata['page']
        
        # 새 Document 객체 생성
        pinecone_docs.append(
            Document(
                page_content=doc.page_content,
                metadata=simplified
            )
        )
    
    # 결과 요약
    if pinecone_docs:
        print(f"변환된 문서 수: {len(pinecone_docs)}")
        print("샘플 메타데이터:")
        print(pinecone_docs[0].metadata)
    
    return pinecone_docs

In [12]:
# 함수 호출 및 문서 로딩
docs = load_and_process_docs(FILE_PATH, EMBED_MODEL_ID)

로드된 문서 수: 13
변환된 문서 수: 13
샘플 메타데이터:
{'headings': ['/cid01173/cid03134/cid02299/cid03295/cid00018'], 'doc_items_labels': ['table'], 'filename': 'prompt_test.pdf'}


In [29]:
docs[2].page_content

"(2) 저출산에 따른 인구구조 변화\n○ 저출산은 학령인구 및 생산연령인구 감소를 초래해 인력 부족, 지역 공동화 등 국가 성장 잠재력 약화로 이어질 우려\n※ 고등학교 인구(15~17세)는 '20년 139만 명에서 '30년 132만 명, '40년 70만 명, '70년 62만 명까지 감소 예측(통계청, 2021)\n○ 모든 학생의 잠재력과 역량을 키워주는 교육 체제 구현을 통한 국가 경쟁력 강화 및 지역 혁신 기반 마련 필요"

- `extract_and_export_tables`: 테이블 추출 함수
    - PDF에 있는 테이블을 csv로 저장하는 함수

In [30]:
import time
from pathlib import Path
import pandas as pd

from docling.document_converter import DocumentConverter

def extract_and_export_tables(input_path, output_directory):
    """
    PDF 파일에서 테이블을 추출하고 CSV로 저장하는 함수
    
    Args:
        input_path (str): 입력 PDF 파일 경로
        output_directory (str): 출력 CSV 파일을 저장할 디렉토리
    
    Returns:
        int: 추출한 표의 수
    """
    # 출력 디렉토리를 Path 객체로 변환
    output_directory = Path(output_directory)
    output_directory.mkdir(parents=True, exist_ok=True)  # 디렉토리 생성
    
    # 입력 파일 경로를 Path 객체로 변환
    input_path = Path(input_path)
    doc_filename = input_path.stem  # 파일명에서 확장자 제외한 부분
    
    # 문서 변환기 초기화
    doc_converter = DocumentConverter()
    
    # 변환 시작 시간 기록
    start_time = time.time()
    print(f"문서 변환 시작: {input_path}")
    
    # 문서 변환
    conv_res = doc_converter.convert(input_path)
    
    # 표 내보내기
    for table_ix, table in enumerate(conv_res.document.tables):
        table_df = table.export_to_dataframe()
        print(f"## Table {table_ix}")
        print(table_df.to_markdown(index=False))  # 표를 마크다운 형식으로 출력
        
        # 표를 csv로 저장
        csv_filename = output_directory / f"{doc_filename}-table-{table_ix+1}.csv"
        print(f"CSV 파일 저장: {csv_filename}")
        table_df.to_csv(csv_filename, index=False)
            
    # 변환 소요 시간 출력
    elapsed_time = time.time() - start_time
    print(f"변환 소요 시간: {elapsed_time:.2f}초")
    
    # 추출한 표 수 확인
    table_count = len(conv_res.document.tables)
    print(f"추출한 표 수: {table_count}")
        
    return table_count
    
# 표 추출 및 내보내기 함수 호출
OUTPUT_DIR = '../create_dataset/source_data/tables'
INPUT_PATH = '../create_dataset/source_data/json 완성 PDF/운영1.pdf'

try:
    table_count = extract_and_export_tables(INPUT_PATH, OUTPUT_DIR)
    print(f"총 {table_count}개의 표가 추출되었습니다.")
except Exception as e:
    print(f"오류 발생: {e}")

문서 변환 시작: ../create_dataset/source_data/json 완성 PDF/운영1.pdf
## Table 0
| 구 분                    | 절대평가.원 점수   | 절대평가.성취도   | 상대평가.석차 등급   | 통계 정보.성취도별 분포 비율   | 통계 정보.과목 평균   | 통계 정보.수강자 수   |
|:-------------------------|:-------------------|:------------------|:---------------------|:-------------------------------|:----------------------|:----------------------|
| 보통 교과                | ○                  | A ･ B ･ C ･ D ･ E | 5등급                | ○                              | ○                     | ○                     |
| 사회 ･ 과학 융합 선택    | ○                  | A ･ B ･ C ･ D ･ E | -                    | ○                              | ○                     | ○                     |
| 체육 ･ 예술/과학탐구실험 | -                  | A ･ B ･ C         | -                    | -                              | -                     | -                     |
| 교양                     | -                  | P                 | -                    | -                              | -          

## 02. Pinecone Upload
- Index 생성
- web에서 직접 index 생성하는 것을 추천함.
    - https://app.pinecone.io/
- 문서를 올릴 때는 반드시 한 문서씩 업로드 (청킹 확인 후 업로드 권장)

In [4]:
from pinecone import Pinecone

# Pinecone API 키와 환경 설정
pc = Pinecone(api_key=pinecone_api_key)
pc_index = pc.Index("mypolio-curriculums")

In [ ]:
# Pinecone 인덱스 확인
pc.list_indexes()

In [7]:
from langchain.embeddings import OpenAIEmbeddings

# OpenAI 임베딩 인스턴스 생성
embeddings = OpenAIEmbeddings(
    model='text-embedding-3-large',
    openai_api_key=openai_api_key
)

/var/folders/jn/6v9ffl5x6gvf4y16kqq_wqgh0000gn/T/ipykernel_24779/588479685.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


In [8]:
from tqdm import tqdm  # 진행 상황을 보여주기 위한 라이브러리
from langchain_pinecone import PineconeVectorStore


# PineconeVectorStore 인스턴스 생성 및 문서 추가
pinecone_database = PineconeVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    index_name="mypolio-curriculums",
    namespace="curriculum", # 네임스페이스 설정: 고교학점제 -> curriculum, 진로&진학 상담 -> course, 고교학점제 테이블 -> curriculum-table, 진로&진학 상담 테이블 -> course-table
)

In [13]:
# 생성된 객체에 대해 문서 추가
pinecone_database.add_documents(
    documents=docs,
    index_name="mypolio-curriculums",
    namespace="curriculum-table", # 네임스페이스 설정: 고교학점제 -> curriculum, 진로&진학 상담 -> course, 고교학점제 테이블 -> curriculum-table, 진로&진학 상담 테이블 -> course-table
)

['368b5093-15af-4989-87fd-25b4cbd4a93f',
 'f4f8bd6b-6ff0-4618-adb7-92febb3e6db4',
 '8e58c8da-641a-4615-99af-20cb16b46e47',
 'cc897b7b-af27-4bca-897f-3a54be60c9ab',
 '2252b2d3-1130-4982-b37f-771f33997bd3',
 'dafc72e2-036e-432e-a836-35db6d7a5fa4',
 'd04d6a35-a430-4a00-a313-cd17cca42603',
 'd2ba48aa-8ba2-4798-bd01-8e85c584bc1c',
 '8351e603-7217-4b37-9c53-a0cc9338eead',
 'cd59edf8-974a-49e1-a8e2-d078b314f254',
 '2a32f597-b9e8-4c14-a861-a2910a8a0cc1',
 'bb952de4-2f14-4989-88b5-446aecb0a807',
 '5c77bb2b-d95a-4bec-a0a5-c4ab9271ce98']

## 03. Pinecone Delete

아래 코드는 반드시 필요한 경우에만 사용합니다.
- 파인콘에 저장한 문서들을 모두 삭제하는 코드

In [16]:
pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index("mypolio-curriculums")

# 특정 네임스페이스의 모든 레코드 삭제
index.delete(delete_all=True, namespace="curriculum")

{}